In [ ]:
api_key_predict= "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"
api_key_analyze= "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"

In [8]:
import requests

invoke_url = "https://ai.api.nvidia.com/v1/cosmos/nvidia/cosmos-predict1-7b"
fetch_url_format = "https://api.nvcf.nvidia.com/v2/nvcf/pexec/status/"

headers = {
    "Authorization": "Bearer $NVIDIA_API_KEY",
    "Accept": "application/json",
}

payload = {
  "inputs": [
    {
      "name": "command",
      "shape": [1],
      "datatype": "BYTES",
      "data": [
        "text2world --prompt=\"A first person view from the perspective from a human sized robot as it works in a chemical plant. The robot has many boxes and supplies nearby on the industrial shelves. The camera on moving forward, at a height of 1m above the floor. Photorealistic\""
      ]
    }
  ],
  "outputs": [
    {
      "name": "status",
      "datatype": "BYTES",
      "shape": [1]
    }
  ]
}

# re-use connections
session = requests.Session()

response = session.post(invoke_url, headers=headers, json=payload)

while response.status_code == 202:
    request_id = response.headers.get("NVCF-REQID")
    fetch_url = fetch_url_format + request_id
    response = session.get(fetch_url, headers=headers)
response = requests.post(invoke_url, headers=headers, json=payload)

response.raise_for_status()

with open('result.zip', 'wb') as f:
  f.write(response.content)

HTTPError: 401 Client Error: Unauthorized for url: https://ai.api.nvidia.com/v1/cosmos/nvidia/cosmos-predict1-7b

In [6]:
import os
import uuid
import requests
import zipfile
import tempfile
import json
import time
from datetime import datetime
from pathlib import Path
import argparse
import io
import shutil

# API keys for various NVIDIA services
API_KEY_COSMOS = os.environ.get("NVIDIA_COSMOS_API_KEY")
API_KEY_VILA = os.environ.get("NVIDIA_VILA_API_KEY")
API_KEY_COSMOS= "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"
API_KEY_VILA= "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"

# API endpoints
COSMOS_URL = "https://ai.api.nvidia.com/v1/cosmos/nvidia/cosmos-predict1-7b"
VILA_URL = "https://ai.api.nvidia.com/v1/vlm/nvidia/vila"
NVCF_ASSET_URL = "https://api.nvcf.nvidia.com/v2/nvcf/assets"
EXEC_STATUS_URL = "https://api.nvcf.nvidia.com/v2/nvcf/pexec/status/"

# Supported media formats
SUPPORTED_FORMATS = {
    "png": ["image/png", "img"],
    "jpg": ["image/jpg", "img"],
    "jpeg": ["image/jpeg", "img"],
    "mp4": ["video/mp4", "video"],
}

def get_extension(filename):
    """Extract file extension from filename."""
    _, ext = os.path.splitext(filename)
    ext = ext[1:].lower()
    return ext

def mime_type(ext):
    """Get MIME type for file extension."""
    return SUPPORTED_FORMATS[ext][0]

def media_type(ext):
    """Get media type (img/video) for file extension."""
    return SUPPORTED_FORMATS[ext][1]

def upload_asset(media_file, api_key, description="Media file for processing"):
    """Upload a media asset to NVIDIA's asset storage."""
    ext = get_extension(media_file)
    if ext not in SUPPORTED_FORMATS:
        raise ValueError(f"Unsupported file format: {ext}")
    
    with open(media_file, "rb") as data_input:
        # Request upload authorization
        headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json",
            "Accept": "application/json",
        }
        
        response = requests.post(
            NVCF_ASSET_URL,
            headers=headers,
            json={"contentType": mime_type(ext), "description": description},
            timeout=30,
        )
        response.raise_for_status()
        auth_data = response.json()
        
        # Upload the file to the provided URL
        upload_response = requests.put(
            auth_data["uploadUrl"],
            data=data_input,
            headers={
                "x-amz-meta-nvcf-asset-description": description,
                "content-type": mime_type(ext),
            },
            timeout=300,
        )
        upload_response.raise_for_status()
        
        if upload_response.status_code == 200:
            print(f"Successfully uploaded asset {auth_data['assetId']}")
        else:
            print(f"Failed to upload asset {auth_data['assetId']}")
        
        return str(uuid.UUID(auth_data["assetId"]))

def delete_asset(asset_id, api_key):
    """Delete an asset from NVIDIA's asset storage."""
    headers = {
        "Authorization": f"Bearer {api_key}",
    }
    
    response = requests.delete(
        f"{NVCF_ASSET_URL}/{asset_id}", 
        headers=headers, 
        timeout=30
    )
    response.raise_for_status()
    print(f"Deleted asset {asset_id}")

def generate_video_from_image(image_path, prompt, api_key):
    """Generate a video from an image using Cosmos Predict1-7B."""
    print(f"Generating video from {image_path} with prompt: {prompt}")
    
    # Upload the image as an asset
    asset_id = upload_asset(image_path, api_key, "Input image for video generation")
    
    # Prepare the API request for video generation
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Accept": "application/json",
    }
    
    payload = {
        "inputs": [
            {
                "name": "command",
                "shape": [1],
                "datatype": "BYTES",
                "data": [
                    f"video2world --image_asset_id={asset_id} --prompt=\"{prompt}\""
                ]
            }
        ],
        "outputs": [
            {
                "name": "status",
                "datatype": "BYTES",
                "shape": [1]
            }
        ]
    }
    
    # Make the API request
    session = requests.Session()
    response = session.post(COSMOS_URL, headers=headers, json=payload)
    
    # Poll for completion if the request is accepted but not yet processed
    request_id = None
    if response.status_code == 202:
        request_id = response.headers.get("NVCF-REQID")
        print(f"Request accepted, polling for completion. Request ID: {request_id}")
        
        while response.status_code == 202:
            time.sleep(10)  # Wait before checking status again
            fetch_url = EXEC_STATUS_URL + request_id
            response = session.get(fetch_url, headers=headers)
            print(f"Status: {response.status_code}")
    
    # Handle the response
    if response.status_code == 200:
        # Save the generated video
        output_path = f"output_{Path(image_path).stem}_{uuid.uuid4().hex[:8]}.zip"
        with open(output_path, 'wb') as f:
            f.write(response.content)
        
        # Extract the video from the ZIP file
        video_path = extract_video_from_zip(output_path)
        
        # Clean up
        delete_asset(asset_id, api_key)
        os.remove(output_path)
        
        return video_path
    else:
        print(f"Error generating video: {response.status_code}")
        print(response.text)
        delete_asset(asset_id, api_key)
        return None

def extract_video_from_zip(zip_path):
    """Extract the video file from the ZIP archive returned by Cosmos."""
    with tempfile.TemporaryDirectory() as tmp_dir:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(tmp_dir)
        
        # Find the video file (usually named out.mp4)
        for root, _, files in os.walk(tmp_dir):
            for file in files:
                if file.endswith('.mp4'):
                    video_path = os.path.join(root, file)
                    output_path = f"video_{uuid.uuid4().hex[:8]}.mp4"
                    shutil.copy(video_path, output_path)
                    return output_path
    
    return None

def analyze_videos_with_vila(video_paths, api_key):
    """Analyze multiple videos using VILA to determine order and relationships."""
    if not video_paths:
        return []
    
    # Upload all videos as assets
    asset_ids = []
    for video_path in video_paths:
        asset_id = upload_asset(video_path, api_key, "Video for analysis")
        asset_ids.append(asset_id)
    
    # Prepare media content string for the query
    media_content = ""
    for asset_id in asset_ids:
        media_content += f'<video src="data:video/mp4;asset_id,{asset_id}" />'
    
    # Make API request to VILA
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
        "NVCF-INPUT-ASSET-REFERENCES": ",".join(asset_ids),
        "NVCF-FUNCTION-ASSET-IDS": ",".join(asset_ids),
        "Accept": "application/json",
    }
    
    query = "Analyze these videos and determine their temporal sequence. Identify any commonalities, events, or narrative flow. Rank them in the most logical order for a coherent story, and briefly explain why."
    
    payload = {
        "max_tokens": 1024,
        "temperature": 0.2,
        "top_p": 0.7,
        "seed": 50,
        "num_frames_per_inference": 8,
        "messages": [
            {
                "role": "user",
                "content": f"{query} {media_content}",
            }
        ],
        "stream": False,
        "model": "nvidia/vila",
    }
    
    response = requests.post(VILA_URL, headers=headers, json=payload)
    
    # Clean up assets
    for asset_id in asset_ids:
        delete_asset(asset_id, api_key)
    
    if response.status_code == 200:
        analysis = response.json()
        print("VILA Analysis:")
        print(analysis)
        
        # Parse the analysis to determine the best sequence
        # This is a simplified approach - in production, you'd want more robust parsing
        try:
            content = analysis["choices"][0]["message"]["content"]
            
            # Use the analysis to determine the best sequence
            # For this example, we'll just return the same order for simplicity
            # In a real application, you would parse the content to determine the optimal order
            return video_paths
        except (KeyError, IndexError) as e:
            print(f"Error parsing VILA response: {e}")
            return video_paths
    else:
        print(f"Error from VILA: {response.status_code}")
        print(response.text)
        return video_paths

def combine_videos(video_paths, output_path="final_story.mp4"):
    """
    Combine multiple videos into a single video.
    
    Note: This function requires ffmpeg to be installed.
    """
    if not video_paths:
        return None
    
    # Create a text file listing all input videos
    with tempfile.NamedTemporaryFile('w', suffix='.txt', delete=False) as f:
        list_file = f.name
        for video_path in video_paths:
            f.write(f"file '{os.path.abspath(video_path)}'\n")
    
    # Use ffmpeg to concatenate the videos
    import subprocess
    try:
        cmd = [
            'ffmpeg',
            '-f', 'concat',
            '-safe', '0',
            '-i', list_file,
            '-c', 'copy',
            output_path
        ]
        subprocess.run(cmd, check=True)
        
        # Clean up the temporary file
        os.unlink(list_file)
        
        return output_path
    except (subprocess.SubprocessError, FileNotFoundError) as e:
        print(f"Error combining videos: {e}")
        if os.path.exists(list_file):
            os.unlink(list_file)
        return None

def generate_final_story(combined_video_path, api_key, prompt="Create a cohesive story based on these events"):
    """Generate a final, enhanced story video from the combined video."""
    if not os.path.exists(combined_video_path):
        return None
    
    # Upload the combined video
    asset_id = upload_asset(combined_video_path, api_key, "Combined video for final generation")
    
    # Prepare the API request
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Accept": "application/json",
    }
    
    payload = {
        "inputs": [
            {
                "name": "command",
                "shape": [1],
                "datatype": "BYTES",
                "data": [
                    f"video2world --video_asset_id={asset_id} --prompt=\"{prompt}\""
                ]
            }
        ],
        "outputs": [
            {
                "name": "status",
                "datatype": "BYTES",
                "shape": [1]
            }
        ]
    }
    
    # Make the API request
    session = requests.Session()
    response = session.post(COSMOS_URL, headers=headers, json=payload)
    
    # Poll for completion
    if response.status_code == 202:
        request_id = response.headers.get("NVCF-REQID")
        print(f"Final generation started. Request ID: {request_id}")
        
        while response.status_code == 202:
            time.sleep(10)
            fetch_url = EXEC_STATUS_URL + request_id
            response = session.get(fetch_url, headers=headers)
    
    # Process the response
    if response.status_code == 200:
        output_path = f"final_story_{datetime.now().strftime('%Y%m%d_%H%M%S')}.zip"
        with open(output_path, 'wb') as f:
            f.write(response.content)
        
        final_video = extract_video_from_zip(output_path)
        
        # Clean up
        delete_asset(asset_id, api_key)
        os.remove(output_path)
        
        return final_video
    else:
        print(f"Error in final generation: {response.status_code}")
        print(response.text)
        delete_asset(asset_id, api_key)
        return None

def main():
    """Main function to run the Experience Story Generator."""
    parser = argparse.ArgumentParser(description="Generate a story video from multiple photos or videos")
    parser.add_argument("--input", "-i", nargs="+", required=True, help="Input media files (images or videos)")
    parser.add_argument("--prompt", "-p", type=str, default="Create a cohesive story from these moments", 
                        help="Prompt for video generation")
    parser.add_argument("--output", "-o", type=str, default="final_story.mp4", 
                        help="Output video filename")
    parser.add_argument("--cosmos-key", type=str, help="NVIDIA Cosmos API key")
    parser.add_argument("--vila-key", type=str, help="NVIDIA VILA API key")
    
    args = parser.parse_args()
    
    # Validate input files (maximum 5)
    input_files = args.input[:5]  # Limit to 5 files
    valid_files = []
    
    for file_path in input_files:
        if not os.path.exists(file_path):
            print(f"Warning: File {file_path} does not exist and will be skipped.")
            continue
            
        ext = get_extension(file_path)
        if ext not in SUPPORTED_FORMATS:
            print(f"Warning: File {file_path} has unsupported format {ext} and will be skipped.")
            continue
            
        valid_files.append(file_path)
    
    if not valid_files:
        print("Error: No valid input files provided.")
        return
    
    # Set API keys
    global API_KEY_COSMOS, API_KEY_VILA
    if args.cosmos_key:
        API_KEY_COSMOS = args.cosmos_key
    if args.vila_key:
        API_KEY_VILA = args.vila_key
    
    if not API_KEY_COSMOS:
        print("Error: NVIDIA Cosmos API key is required. Set it with --cosmos-key or export NVIDIA_COSMOS_API_KEY.")
        return
    
    if not API_KEY_VILA:
        print("Error: NVIDIA VILA API key is required. Set it with --vila-key or export NVIDIA_VILA_API_KEY.")
        return
    
    # Process each input file to generate a video
    generated_videos = []
    for file_path in valid_files:
        # Generate a custom prompt based on the file name
        file_prompt = f"{args.prompt} Scene depicted in this {get_extension(file_path)}."
        
        video_path = generate_video_from_image(file_path, file_prompt, API_KEY_COSMOS)
        if video_path:
            generated_videos.append(video_path)
    
    if not generated_videos:
        print("Error: Failed to generate any videos.")
        return
    
    # Analyze and reorder videos using VILA
    ordered_videos = analyze_videos_with_vila(generated_videos, API_KEY_VILA)
    
    # Combine the videos
    combined_path = combine_videos(ordered_videos, "intermediate_combined.mp4")
    if not combined_path:
        print("Error: Failed to combine videos.")
        return
    
    # Generate the final, enhanced story video
    final_story = generate_final_story(combined_path, API_KEY_COSMOS, args.prompt)
    if final_story:
        # Rename to the user-specified output path
        if os.path.exists(args.output):
            os.remove(args.output)
        os.rename(final_story, args.output)
        print(f"Success! Final story video created at: {args.output}")
        
        # Clean up intermediate files
        for video in generated_videos:
            if os.path.exists(video):
                os.remove(video)
        if os.path.exists(combined_path):
            os.remove(combined_path)
    else:
        print("Error: Failed to generate final story video.")

if __name__ == "__main__":
    main()

usage: ipykernel_launcher.py [-h] --input INPUT [INPUT ...] [--prompt PROMPT]
                             [--output OUTPUT] [--cosmos-key COSMOS_KEY]
                             [--vila-key VILA_KEY]
ipykernel_launcher.py: error: the following arguments are required: --input/-i


SystemExit: 2

/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:

import requests
import os
import uuid
import sys

invoke_url = "https://ai.api.nvidia.com/v1/vlm/nvidia/vila"
stream = False
query = "Describe the scene"

kApiKey = os.getenv("TEST_NVCF_API_KEY", "")
assert kApiKey, "Generate API_KEY and export TEST_NVCF_API_KEY=xxxx"

kNvcfAssetUrl = "https://api.nvcf.nvidia.com/v2/nvcf/assets"
# ext: {mime, media}
kSupportedList = {
    "png": ["image/png", "img"],
    "jpg": ["image/jpg", "img"],
    "jpeg": ["image/jpeg", "img"],
    "mp4": ["video/mp4", "video"],
}

def get_extention(filename):
    _, ext = os.path.splitext(filename)
    ext = ext[1:].lower()
    return ext

def mime_type(ext):
    return kSupportedList[ext][0]
def media_type(ext):
    return kSupportedList[ext][1]

def _upload_asset(media_file, description):
    ext = get_extention(media_file)
    assert ext in kSupportedList
    data_input = open(media_file, "rb")
    headers={
        "Authorization": f"Bearer {kApiKey}",
        "Content-Type": "application/json",
        "accept": "application/json",
    }
    assert_url = kNvcfAssetUrl
    authorize = requests.post(
        assert_url,
        headers = headers,
        json={"contentType": f"{mime_type(ext)}", "description": description},
        timeout=30,
    )
    authorize.raise_for_status()

    authorize_res = authorize.json()
    print(f"uploadUrl: {authorize_res['uploadUrl']}")
    response = requests.put(
        authorize_res["uploadUrl"],
        data=data_input,
        headers={
            "x-amz-meta-nvcf-asset-description": description,
            "content-type": mime_type(ext),
        },
        timeout=300,
    )

    response.raise_for_status()
    if response.status_code == 200:
        print(f"upload asset_id {authorize_res['assetId']} successfully!")
    else:
        print(f"upload asset_id {authorize_res['assetId']} failed.")
    return uuid.UUID(authorize_res["assetId"])

def _delete_asset(asset_id):
    headers = {
        "Authorization": f"Bearer {kApiKey}",
    }
    assert_url = f"{kNvcfAssetUrl}/{asset_id}"
    response = requests.delete(
        assert_url, headers=headers, timeout=30
    )
    response.raise_for_status()

def chat_with_media_nvcf(infer_url, media_files, query: str, stream: bool = False):
    asset_list = []
    ext_list = []
    media_content = ""
    assert isinstance(media_files, list), f"{media_files}"
    print("uploading {media_files} into s3")
    has_video = False
    for media_file in media_files:
        ext = get_extention(media_file)
        assert ext in kSupportedList, f"{media_file} format is not supported"
        if media_type(ext) == "video":
            has_video = True
        asset_id = _upload_asset(media_file, "Reference media file")
        asset_list.append(f"{asset_id}")
        ext_list.append(ext)
        media_content += f'<{media_type(ext)} src="data:{mime_type(ext)};asset_id,{asset_id}" />'
    if has_video:
        assert len(media_files) == 1, "Only single video supported."
    asset_seq = ",".join(asset_list)
    print(f"received asset_id list: {asset_seq}")
    headers = {
        "Authorization": f"Bearer {kApiKey}",
        "Content-Type": "application/json",
        "NVCF-INPUT-ASSET-REFERENCES": asset_seq,
        "NVCF-FUNCTION-ASSET-IDS": asset_seq,
        "Accept": "application/json",
    }
    if stream:
        headers["Accept"] = "text/event-stream"
    response = None

    messages = [
        {
            "role": "user",
            "content": f"{query} {media_content}",
        }
    ]
    payload = {
        "max_tokens": 1024,
        "temperature": 0.2,
        "top_p": 0.7,
        "seed": 50,
        num_frames_per_inference: 8,
        "messages": messages,
        "stream": stream,
        "model": "nvidia/vila",
    }

    response = requests.post(infer_url, headers=headers, json=payload, stream=stream)
    if stream:
        for line in response.iter_lines():
            if line:
                print(line.decode("utf-8"))
    else:
        print(response.json())

    print(f"deleting assets: {asset_list}")
    for asset_id in asset_list:
        _delete_asset(asset_id)

if __name__ == "__main__":
    """ export TEST_NVCF_API_KEY=xxx.
        python test.py sample.mp4
        python test.py sample1.png sample2.png
    """

    if len(sys.argv) <= 1:
        print("Usage: export TEST_NVCF_API_KEY=xxx")
        print(f"       python {sys.argv[0]} sample1.png sample2.png ... sample16.png")
        print(f"       python {sys.argv[0]} sample.mp4")
        sys.exit(1)
    media_samples = list(sys.argv[1:])
    chat_with_media_nvcf(invoke_url, media_samples, query, stream)


AssertionError: Generate API_KEY and export TEST_NVCF_API_KEY=xxxx